# Load modules

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go


``lib.volume`` contains helper fucntions and ``Volume`` class.

In [2]:
import lib.volume as vl

# Load Data

In [3]:
Synthetic_FaultSegments = np.load("data/Synthetic_FaultSegments.npy")
Synthetic_FullStack = np.load("data/Synthetic_FullStack.npy")


Prepare instance of ``Volume`` and create 3-D points that originate from the fault segments

In [4]:
volume = vl.Volume(Synthetic_FullStack, Synthetic_FaultSegments)

In [5]:
data, prob = volume.get_segm_points(0.5)

Interactive visualization of the extracted points.

In [6]:
fig = go.Figure(data=[
    go.Scatter3d(
        x= data[::160,0], 
        y= data[::160,1],
        z=-data[::160,2],
        mode='markers',
        marker={
            'size': 1.5,
            'color': data[::160,2]
        }
    ),
],
    layout = go.Layout(
        height=1000,
        scene = dict(
            aspectmode='data',
    )
))
fig.show()

# Process extracted points

Fault extraction is based on the clustering of the points. 
Since each fault can be represented as a region with high density of points, density-based clutering algorithms should suit well to acomplish the task of fault detection/extraction.
As a clustering algorithm DBSCAN has been selected. Alternative is OPTICS algorithm. In current analyzis, DBSCAN showed faster execution times compared to OPTICS algorithm.


Configuration parameters of the DBSCAN that are used for fault detection:
- min number of points to form a core: 19
- max neighbour distance: 3

In [7]:
labels_faults = vl.get_fault_clusters(data, min_samples=19, eps=3)

DBSCAN generates labels for each points; points with the same label belong to the same cluster (fault)

In [ ]:
fig = go.Figure(data=[
    go.Scatter3d(
        x= data[::160,0], 
        y= data[::160,1],
        z=-data[::160,2],
        mode='markers',
        marker={
            'size': 1.5,
            'color': labels_faults[::160]
        }
    ),
],
    layout = go.Layout(
        height=800,
        scene = dict(
            aspectmode='data',
    )
))
fig.show()

# Generate csv file

Detected faults are downsampled and stored into csv file.

In [ ]:
# create empty dataFrame
df_fault_points = pd.DataFrame(columns=['Fault','X', 'Y', 'Z'])
#iterate over labels

# min number of points to consider the cluster as a fault
min_npoints = 500
# downsample factor
downsample = 150

for idx in range(np.max(labels_faults)+1):
    # pick all points sharing the label "idx"
    fault = data[labels_faults == idx]
    if len(fault) > min_npoints:
        # iterate over the array of points with stride of downsample
        for p in np.arange(0,len(fault)-1, downsample):
            df_fault_points = df_fault_points.append({'Fault': idx,
                                                      'X': fault[int(p), 0], 
                                                      'Y':fault[int(p), 1],
                                                      'Z':fault[int(p), 2] }, ignore_index=True)
# set correct type of the dataframe
df_fault_points = df_fault_points.astype('float32')


## Process faults as in task 1 

In [ ]:
from lib.fault import *

fault_points_grouped = df_fault_points.groupby('Fault')
faults = [Fault(idx,f[['X','Y','Z']].to_numpy(), 1) for idx, f in fault_points_grouped]


In [ ]:
faults = process_faults(faults)
df_faults = generate_table(faults)

In [ ]:
df_faults.head()

### Visualize extracted faults

In [ ]:
surfs = []
for _, fault in fault_points_grouped:
    surfs.append(go.Mesh3d(x= fault['X'], 
                           y= fault['Y'], 
                           z=-fault['Z'],
                           alphahull=-1,
                          opacity=1
                    ))
    
fig = go.Figure(data=surfs,
                layout = go.Layout(
                height=900,
                scene = dict(
                    aspectmode='data',
                )
    )  
)

fig.show()

## Store the result

### Fault points to CSV file

Store downsampled labeled faul points into CSV file

In [ ]:
df_fault_points.to_csv("results/5-Fault_points.csv")

### Store labels as a 3D volume

Store labels as a volume data. Each point in the volume indicates a label of the fault instance. Label can be -1, 0, or more than 0:
- -1 indicates an empty volume
- 0 indicates a point (voxel) that is not assigned to any of the faults
- fault voxels take labels more than 0 (1,2,...) 

In [ ]:
labeled_volume = np.full(Synthetic_FaultSegments.shape, -1., dtype=np.float32)
labeled_volume[data[:,0], data[:,1], data[:,2]] = labels_faults + 1 # add 1 since DBSCAN has 0-indexed labels for clusters.
#np.save("data/Synthetic_FullStackLabeled_Segment.npy", labeled_volume)
np.save("results/Synthetic_FullStackLabeled_Segment.npy", labeled_volume)
